## Off the shelf results with FLAN

In [48]:
%load_ext autoreload
%autoreload 2
from dataset.privacy_policies_dataset import DatasetLoader
from dataset.templates import PATTERNS

from src.privacyglue_tasks.policy_qa import load_policy_qa
from src.privacyglue_tasks.privacy_qa import load_privacy_qa
from src.utils.task_utils import policy_ie_file_mapping

from transformers import T5ForConditionalGeneration, T5Tokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Privacy Policy Dataset Loading

In [55]:
dataset_loader = DatasetLoader()

  0%|          | 0/158 [00:00<?, ?ba/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/63 [00:00<?, ?ba/s]

Using custom data configuration default-9bff2d11f4c1db3e
Reusing dataset text (/Users/loaynasser/.cache/huggingface/datasets/text/default-9bff2d11f4c1db3e/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/1041 [00:00<?, ?ex/s]

  0%|          | 0/4209 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/1041 [00:00<?, ?ex/s]

In [56]:
dataset_loader.policy_ie_a_dataset["train"][0]["label"]

'Data Storage Retention Deletion'

In [37]:
dataset_loader.piextract_dataset

DatasetDict({
    train: {'COLLECT': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 2579
    }), 'NOT_COLLECT': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 2579
    }), 'NOT_SHARE': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 2579
    }), 'SHARE': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 2579
    })}
    test: {'COLLECT': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 1029
    }), 'NOT_COLLECT': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 1029
    }), 'NOT_SHARE': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 1029
    }), 'SHARE': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 1029
    })}
    validation: {'COLLECT': Dataset({
        features: ['tokens', 'tags', 'subtask'],
        num_rows: 456
    }), 'NOT_COLLECT': Dataset({
     

## Initializing the model & tokenizer

In [59]:
#Small Model & Tokenizer Loading
small_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')
small_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

#Base Model & Tokenizer Loading
base_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
base_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

#Large Model & Tokenizer Loading
large_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')
large_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

xlarge_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xl')
xlarge_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-xl')

xxlarge_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl')
xxlarge_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-xxl')


In [35]:
# prompt for OPP-115 with T5

t5_prepared_text =  dataset_loader.policy_detection_dataset["train"][158]["text"]
input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

# q&a 
opp_115_output_ids = base_model.generate(
    input_ids
)

output = base_tokenizer.decode(opp_115_output_ids[0], skip_special_tokens=True)

print (f"Response: \n{output}")

Response: 
Policy


## OPP-115 --- Zero-Shot Learning
A total of 115 privacy policies were selected based on their corresponding company's popularity on Google Trends. 
Given an input paragraph segment of a privacy policy. the goal is to predict one or more data practive categories.

In [77]:
# prompt for OPP-115 with T5

t5_prepared_text =  dataset_loader.opp_115_dataset['train'][3]['text']
input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

# q&a 
opp_115_output_ids = base_model.generate(
    input_ids
)

output = base_tokenizer.decode(opp_115_output_ids[0], skip_special_tokens=True)

print (f"Response: \n{output}")

/Users/loaynasser/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Response: 
First Party Collection/Use


In [78]:
# Define evaluation metrics

# Prepare evaluation data
dataset = dataset_loader.opp_115_dataset['train']  # or any other appropriate evaluation subset

# Evaluate zero shot predictions
total_examples = len(dataset)

small_correct_predictions = 0
base_correct_predictions = 0
large_correct_predictions = 0
xlarge_correct_predictions = 0
xxlarge_correct_predictions = 0

for example in dataset:
    t5_prepared_text = example['text']
    
    input_ids = large_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    opp_115_output_ids = large_flan_model.generate(input_ids)
    output = large_flan_tokenizer.decode(opp_115_output_ids[0], skip_special_tokens=True)

    # Compare the predicted output with the ground truth label
    large_predicted_label = output  # Modify this line based on your specific use case
    ground_truth_label = example['label']  # Modify this line based on your specific use case

    print("predicted",small_predicted_label,base_predicted_label, large_predicted_label, xlarge_predicted_label, xxlarge_predicted_label)
    print("ground truth",ground_truth_label)
    print("--------------------------------")
    
    if small_predicted_label in ground_truth_label:
        small_correct_predictions += 1
        
    if base_predicted_label in ground_truth_label:
        base_correct_predictions += 1
    
    if large_predicted_label in ground_truth_label:
        large_correct_predictions += 1
        
    if xlarge_predicted_label in ground_truth_label:
        xlarge_correct_predictions += 1
        
    if xxlarge_predicted_label in ground_truth_label:
        xxlarge_correct_predictions += 1
    
        
    

# You can also calculate and print other metrics based on your requirements
# ...


predicted Privacy contact information
ground truth ['First Party Collection/Use']
--------------------------------
predicted Third Party Sharing/Collection
ground truth ['Third Party Sharing/Collection']
--------------------------------
predicted Data Security
ground truth ['Third Party Sharing/Collection', 'First Party Collection/Use']
--------------------------------
predicted First Party Collection/Use
ground truth ['Introductory/Generic']
--------------------------------
predicted First Party Collection/Use
ground truth ['Third Party Sharing/Collection', 'First Party Collection/Use']
--------------------------------
predicted First Party Collection/Use
ground truth ['Introductory/Generic']
--------------------------------
predicted Data Security
ground truth ['User Access, Edit and Deletion']
--------------------------------
predicted First Party Collection/Use
ground truth ['Introductory/Generic']
--------------------------------
predicted First Party Collection/Use
ground truth [

In [79]:
# Calculate evaluation metrics
accuracy = correct_predictions / total_examples

# Print the evaluation results
print(f"Accuracy: {accuracy}")

Accuracy: 0.45720823798627


In [19]:
# pass labels in to calculate loss
input_ids = base_tokenizer(prompts[1],padding=True, truncation=True, return_tensors='pt').input_ids
labels = base_tokenizer(flat_labels[1],padding=True, truncation=True, return_tensors='pt').input_ids

loss = base_model(input_ids=input_ids, labels=labels).loss

labels, loss

(tensor([[ 9879,  3450, 25037,    87,  9939, 12252,     1]]),
 tensor(0.0926, grad_fn=<NllLossBackward0>))

## CoLA: The Corpus of Linguistic Acceptability
checking for grammatical correctess

In [28]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

# CoLA 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")


is grammatically correct?: 
acceptable


In [29]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

# summmarize 
translate_ids = base_model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

is grammatically correct?: 
unacceptable


## STSB - Semantic Text Similarity Benchmark
Are two sentences semantically similar

In [30]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity 
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (0-5): 
3.2


In [31]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (0-5): 
0.4


## MNLI - Multi-Genre Natural Language Inference
Whether a premise implies (“entailment”), contradicts (“contradiction”), or neither (“neutral”) a hypothesis.

In [32]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I am running for mayor', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
entailment


In [33]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I do not really vote', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
contradiction


In [34]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I code for a living', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
neutral


## Q/A - Question/Answering

In [35]:
input_ids = base_tokenizer.encode(
    'question: Where does Sinan live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
California


In [36]:
input_ids = base_tokenizer.encode(
    'question: Where does Matt live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
Boston


In [37]:
# Sanity check with random prompts

input_ids = base_tokenizer.encode(
    'prompt1: Where does Matt live? prompt2: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
prompt2


## Fine-tuning T5 for abstractive summarization

In [25]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random

In [26]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-small')
base_tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [28]:
# https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv

reviews = pd.read_csv('Reviews.csv')

# Pre-processing step
# Punctuation is important in grammar and important for complex decoding architectures to know when to stop!
def add_punc(s):
    if s[-1] not in ('.', '!', '?'):
        s = s + '.'
    return s

reviews.dropna(inplace=True)

reviews['Summary'] = reviews['Summary'].map(add_punc)

print(reviews.shape)

reviews.head()

(568411, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food.,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised.,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all.",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine.,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy.,Great taffy at a great price. There was a wid...


In [29]:
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >= 30)]

reviews.shape

(157786, 10)

In [31]:
random.seed(0)

reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(5000))
reviews_dataset


Dataset({
    features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'],
    num_rows: 5000
})

In [61]:
# We have a prompt but only as a prefix in the encoder
prefix = "summarize: "

# we will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = base_tokenizer(inputs, max_length=1024, truncation=True)

    labels = base_tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [62]:
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [63]:
# inspect a single datapoint
tokenized_reviews_dataset['train'][0]

KeyError: "Column train not in the dataset. Current columns in the dataset: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels']"

In [64]:
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=.1)

In [65]:
# Data collator specifically for generic sequence to sequence tasks
# Use when we are translating one sequence to another like translation, summarization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=base_tokenizer, model=base_model)

In [66]:
training_args = TrainingArguments(
    output_dir="./t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    data_collator=data_collator,
)

trainer.evaluate()

{'eval_loss': 4.56210470199585,
 'eval_runtime': 45.1739,
 'eval_samples_per_second': 11.068,
 'eval_steps_per_second': 0.354}

In [ ]:
trainer.train()

/Users/loaynasser/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.592700,3.335303


In [28]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Id, ProfileName, Summary, Score, Time, __index_level_0__, UserId, ProductId, Text, HelpfulnessDenominator, HelpfulnessNumerator. If Id, ProfileName, Summary, Score, Time, __index_level_0__, UserId, ProductId, Text, HelpfulnessDenominator, HelpfulnessNumerator are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 3.3603246212005615,
 'eval_runtime': 33.7016,
 'eval_samples_per_second': 14.836,
 'eval_steps_per_second': 0.475,
 'epoch': 2.0}

In [29]:
trainer.save_model()

Saving model checkpoint to ./t5_summary_results
Configuration saved in ./t5_summary_results/config.json
Model weights saved in ./t5_summary_results/pytorch_model.bin


In [34]:
loaded_model = T5ForConditionalGeneration.from_pretrained('./t5_summary_results')

# summarization pipeline prepends a default prefix of summarize: 
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

loading configuration file ./t5_summary_results/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_sto

In [52]:
sam = reviews.sample(1)

print(sam['Summary'])

text = sam['Text'].tolist()[0]
text

418718    best beans for the buck you will find.
Name: Summary, dtype: object


'Very happy with this coffee. mellow, and rich, organic and fair trade, what more can you ask for? Oh yeah, what a great deal! THanks'

In [53]:
# Generate a summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'Great coffee.'}]

In [54]:
# Try the base t5 on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-small'
)

loading configuration file config.json from cache at /Users/sinanozdemir/.cache/huggingface/hub/models--t5-small/snapshots/9507060efcd5189100109e25df8326eb07274a36/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram

In [55]:
# Summary is a bit more extractive than our fine-tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'THanks coffee is a great coffee . mel'}]